<a href="https://colab.research.google.com/github/leegyuhi/oooooooooooooh/blob/master/Seq2seq_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. Sequence to Sequence
시퀀스-투-시퀀스(Sequence-to-Sequence)는 입력된 시퀀스로부터 다른 도메인의 시퀀스를 출력하는 다양한 분야에서 사용되는 모델입니다. 예를 들어 챗봇(Chatbot)과 기계 번역(Machine Translation)이 그러한 대표적인 예인데, 입력 시퀀스와 출력 시퀀스를 각각 질문과 대답으로 구성하면 챗봇으로 만들 수 있고, 입력 시퀀스와 출력 시퀀스를 각각 입력 문장과 번역 문장으로 만들면 번역기로 만들 수 있습니다.

![seq2seq_01](https://user-images.githubusercontent.com/52767505/70041761-11f0e600-1601-11ea-9681-ac346c0e8b88.png)

위의 그림은 seq2seq 모델로 만들어진 번역기가 'I love you'라는 영어 문장을 받아서, 'Nan nul saranghey(난 널 사랑해)'라는 한국어 문장을 출력하는 모습을 보여줍니다. 지금부터 seq2seq 모델 내부 구성을 알아보겠습니다.

![seq2seq_02](https://user-images.githubusercontent.com/52767505/70046263-01dd0480-1609-11ea-92bc-42a60db7e879.png)

seq2seq는 크게 인코더와 디코더로 구성되어 있습니다. 인코더는 입력 문장의 모든 단어들을 순차적으로 입력받은 뒤에 마지막에 이 모든 단어 정보들을 압축해서 하나의 벡터로 만듭니다. 이 벡터를 context vector라고 합니다. 입력 문장의 정보가 하나의 context vector로 모두 압축되면, 이 벡터를 디코더로 전송합니다. 디코더는 context vector를 받아서 번역된 단어를 한 개씩 순차적으로 출력합니다.

![seq2seq_03](https://user-images.githubusercontent.com/52767505/70046264-01dd0480-1609-11ea-9816-bb4887e25e92.png)

인코더와 디코더는 각각 여러개의 RNN 셀로 구성되어 있습니다. 입력 문장을 받는 RNN 셀을 인코더라고 하고, 출력 문장을 내보내는 RNN 셀을 디코더라고 합니다. 일반적으로 RNN 셀의 성능문제 때문에 LSTM 셀이나 GRU 셀을 사용합니다. 

우선 인코더에서 입력 문장은 단어 단위로 쪼개져서 RNN 셀 각각의 입력이 됩니다. 인코더의 마지막 RNN 셀의 은닉 상태를 디코더의 RNN 셀로 넘겨주는데, 이를 컨텍스트 벡터라고 합니다. 이렇게 만들어진 컨텍스트 벡터는 앞에서 받은 모든 입력 값의 데이터를 함축적으로 가지고 있습니다. 컨텍스트 벡터는 다시 디코더 RNN 셀의 첫번째 은닉 상태로 사용됩니다.

컨텍스트 벡터를 받은 디코더는 초기 입력으로 문장의 시작을 의미하는 <start> 심볼을 받습니다. 디코더가 <start> 심볼을 받으면 현재 시점에서 등장할 확률이 높은 단어를 예측합니다. 디코더의 첫번째 RNN 셀은 예측된 단어를 다음 시점의 RNN 셀의 입력으로 보냅니다. 그다음 RNN 셀은 입력된 단어로부터 다시 다음에 올 단어를 예측합니다. 디코더는 이런 식으로 입력된 단어의 다음에 올 단어를 예측하고 다음 시점의 RNN 셀의 입력으로 넣는 과정을 반복합니다. 문장의 끝을 의미하는 <end> 심볼이 다음 단어로 예측될 때 까지 이 과정을 반복합니다.

![seq2seq_04](https://user-images.githubusercontent.com/52767505/70048945-da893600-160e-11ea-9f7d-9eaafbd4fd18.png)

인코더에서 각각의 RNN 셀이 받은 입력을 어떻게 다음 RNN 셀로 넘기는지 좀 더 자세히 알아보도록 하겠습니다. 먼저 인코더의 RNN 셀에서 입력을 받으면 은닉층으로 이동하여 은닉 상태와 이전에 받은 입력을 합쳐서 인코더로 피드백합니다. 이러한 과정을 반복하여 인코더의 마지막 RNN 셀에서 컨텍스트 벡터를 만들어서 디코더로 보냅니다.

# 참고문헌
- Dzmitry Bahdanau, KyungHyun Cho, Yoshua Bengio, "Neural Machine Translation by Jointly Learning to Aligh and Translate", 2015
- "How to Use Tensorflow for Seq2seq Models (LIVE)", Youtube, 2017.3.30, https://www.youtube.com/watch?v=ElmBrKyMXxs
- Won Joon Yoo, [딥 러닝을 이용한 자연어 처리 입문], BY-NC-SA, 2019.12.03, 13-1)시퀀스-투-시퀀스